In [62]:
import pandas as pd

In [63]:
def extract_excels():
    try:
        path =  'preprocessing/preprocessed files/'

        df_Airbnb = pd.read_excel(path + 'Final_Airbnb.xlsx')
        print("df_Airbnb succesfully loaded!")   

        df_Booking = pd.read_excel(path + 'Final_Booking.xlsx')
        print("df_Booking succesfully loaded!")  

        df_Oltp = pd.read_excel(path + 'Final_Oltp.xlsx')
        print("df_Oltp succesfully loaded!")

    except Exception as err:
        print('Error while connecting to the db')
        print(err) 

    return df_Airbnb, df_Booking, df_Oltp

In [64]:
df_Airbnb, df_Booking, df_Oltp = extract_excels()

df_Airbnb succesfully loaded!
df_Booking succesfully loaded!
df_Oltp succesfully loaded!


In [65]:
df_merged = pd.concat([df_Airbnb, df_Booking, df_Oltp]).sort_values("Start date")
df_merged.head()

,Unnamed: 0,Status,Guest name,Origin,# of guests,Booked,Start date,End date,# of nights,# of days pre booked,Listing name,# of beds,Earnings,Tax,Earnings after Tax,Advertiser
1,1,Confirmed,Youstina Daoud,Egypt,2,2019-03-30,2019-06-03,2019-06-05,2,65,Studio with Patio,2,67.90,4.0,63.90,Airbnb
6,6,Confirmed,Kristina Holm Jensen,Denmark,2,2019-05-05,2019-06-05,2019-06-07,2,31,Studio with Patio,2,67.90,4.0,63.90,Airbnb
2,2,Confirmed,Öznur Balaban,Turkey,2,2019-04-09,2019-06-08,2019-06-09,1,60,Studio with Patio,2,33.95,2.0,31.95,Airbnb
10,10,Confirmed,Steve Qj,United States,1,2019-06-01,2019-06-10,2019-06-11,1,9,Studio with Patio,2,33.95,1.0,32.95,Airbnb
5,5,Confirmed,Quinten Spakman,Netherlands,2,2019-05-04,2019-06-13,2019-06-15,2,40,Studio with Patio,2,67.90,4.0,63.90,Airbnb


## First DW theme

In [66]:
df = pd.DataFrame()

# Initialize empty data frames for required tables
dim_Apartment = pd.concat([df.copy(), pd.DataFrame(columns=['apartment_id', 'apartment_name'])], axis=1)
dim_Advertiser = pd.concat([df.copy(), pd.DataFrame(columns=['advertiser_id', 'advertiser_name'])], axis=1)
dim_Time = pd.concat([df.copy(), pd.DataFrame(columns=['time_id', 'date', 'day', 'month', 'year'])], axis=1)
fct_Cashflow = pd.concat([df.copy(), pd.DataFrame(columns=['cashflow_id', 'apartment_id', 'advertiser_id', 'time_id', 'earnings_for_day'])], axis=1)

Dim_Apartment

In [67]:
df_merged["apartment_id"] = df_merged["Listing name"].rank(method='dense').astype(int)

dim_Apartment = df_merged[['apartment_id', 'Listing name']].drop_duplicates().sort_values("apartment_id")
dim_Apartment.head()


,apartment_id,Listing name
7,1,One-Bedroom Apartment with Balcony and Sea View
22,2,One-Bedroom Apartment with Patio and Sea View
1,3,Studio with Patio
23,4,Studio with Patio and Sea View


Dim_Advertiser

In [68]:
df_merged["advertiser_id"] = df_merged["Advertiser"].rank(method='dense').astype(int)

dim_Advertiser = df_merged[['advertiser_id', 'Advertiser']].drop_duplicates().sort_values("advertiser_id")
dim_Advertiser.head()

,advertiser_id,Advertiser
1,1,Airbnb
22,2,Booking


Dim_Time

Fct_Cashflow

In [69]:
df_reservation = df_merged.loc[df_merged["Status"] == "Confirmed"].copy()
df_reservation["Price per day"] = df_reservation["Earnings after Tax"] / df_reservation["# of nights"]

df_reservation = df_reservation[["Start date", "End date", "apartment_id" , "advertiser_id", "Earnings after Tax", "# of nights", "Price per day"]]
df_reservation['date'] = df_reservation.apply(lambda x: pd.date_range(start=x['Start date'], end=x['End date'] - pd.Timedelta(days=1)), axis=1)
df_reservation.head()


,Start date,End date,apartment_id,advertiser_id,Earnings after Tax,# of nights,Price per day,date
1,2019-06-03,2019-06-05,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-03', '2019-06-04'], dt..."
6,2019-06-05,2019-06-07,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-05', '2019-06-06'], dt..."
2,2019-06-08,2019-06-09,3,1,31.95,1,31.95,"DatetimeIndex(['2019-06-08'], dtype='datetime6..."
10,2019-06-10,2019-06-11,3,1,32.95,1,32.95,"DatetimeIndex(['2019-06-10'], dtype='datetime6..."
5,2019-06-13,2019-06-15,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-13', '2019-06-14'], dt..."


In [70]:
df_reservation = df_reservation.explode('date')
df_reservation = df_reservation.drop(['Start date', 'End date', 'Earnings after Tax', '# of nights'], axis=1)
df_reservation.head()


,apartment_id,advertiser_id,Price per day,date
1,3,1,31.95,2019-06-03
1,3,1,31.95,2019-06-04
6,3,1,31.95,2019-06-05
6,3,1,31.95,2019-06-06
2,3,1,31.95,2019-06-08
